<a href="https://colab.research.google.com/github/plaban1981/Agents/blob/main/LLM_Router.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install -qU langchain
!pip install -qU groq
!pip install -qU langchain-groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 5.5 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [17]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.4/362.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 12.8 MB/s eta 0:00:00


In [18]:
import openai

In [5]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [12]:
def openrouter_completion(model_name, user_query, system_message=None):
    messages = []
    if system_message:
        messages.append({"role": "system", "content": system_message})
    messages.append({"role": "user", "content": user_query})
    completion = client.chat.completions.create(
        model=model_name,
        messages=messages
    )
    return completion.choices[0].message.content

In [14]:
def router(user_input):
    response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[
            {"role": "system", "content": """You are a helpful router designed to output JSON. you decide which route user input belongs to based on the type of the user query
             return your response as follows:
             {{
             "route": "route_name", can be one of the following:
             - "llama-3.1-70b-versatile"- for more complex, in depth questions and queries except code
             - "llama3-70b-8192" for all code queries.
             - "gemma-7b-it" for other simple daily conversation like regular speak
             }}"""},
            {"role": "user", "content": user_input}
        ]
    )
    print(type(response.choices[0].message.content))
    return response.choices[0].message.content

In [15]:
import json
while True:
    user_input = input("Please enter your question (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        break

    # Attempt to parse JSON, handle potential errors
    try:
        response_text = router(user_input)
        model = json.loads(response_text)
        model = model["route"]
    except json.JSONDecodeError:
        print("Error: Invalid JSON returned from router. Please review the router's output.")
        print(f"Router raw output: {response_text}") # Print raw output for debugging
        continue # Skip to next iteration of the loop

    if model == "llama3-8b-8192":
        system_message = "You are a masterful english literary"
        print(openrouter_completion(model, user_input, system_message))
    elif model == "llama-3.1-70b-versatile":
        print(openrouter_completion(model, user_input))
    elif model == "gemma-7b-it":
        print(openrouter_completion(model, user_input))

Please enter your question (or type 'exit' to quit): What is the Contribution of Shakesphere to English Literature ?
<class 'str'>
I think you meant "Shakespeare"!

William Shakespeare (1564-1616) is widely regarded as one of the greatest writers in the English language and the world's pre-eminent dramatist. His contributions to English literature are immense and far-reaching. Here are some of the most significant:

1. **Invention of New Words and Phrases**: Shakespeare introduced many words and phrases that are still in use today. It's estimated that he contributed over 1700 words to the English language, including "assassination," "bedazzled," "fashionable," and "lonely."
2. **Development of the English Sonnet**: Shakespeare popularized the sonnet, a 14-line poem with a specific rhyme scheme. His sonnets are still studied and admired for their beauty, complexity, and emotional depth.
3. **Shaping the English Language**: Shakespeare's writing helped to standardize English spelling and

## Advanced Routing

In [109]:
def openrouter_completion(model_name, user_query, system_message=None):
    messages = []
    if system_message:
        messages.append({"role": "system", "content": system_message})
    messages.append({"role": "user", "content": user_query})
    completion = client.chat.completions.create(
        model=model_name,
        messages=messages,
        stream=True
    )
    print(f"Completions:{completion}")
    assistant_response = ""
    print(colored("Assistant response:", "green"))
    try:

      for chunk in completion:
          if chunk.choices[0].delta.content is not None:
              chunk_content = chunk.choices[0].delta.content
              assistant_response += chunk_content
              print(chunk_content, end="", flush=True)
      return assistant_response
    except:
      return completion.choices[0].message.content

In [139]:
def primary_router(user_input):
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": """You are a helpful router designed to output JSON. You decide which route user input belongs to based on the type of the user query.
             Return your response as follows:
             {"route": "route_name"}
             The route_name can be one of the following:
             - "llama3-70b-8192" for more complex, in-depth questions and queries. This route doesn't accept programming related queries.
             - "llama-3.1-70b-versatile" for all code and programming related queries.
             - "gemma2-9b-it" for other simple daily conversation like regular speak
             - "llama-guard-3-8b" for security related for search and code interpreter tool calls.
             """},
            {"role": "user", "content": user_input}
        ]
    )
    return json.loads(response.choices[0].message.content)["route"]

#### Test the function

In [131]:
primary_router("What is Neural Dust?")

'gemma2-9b-it'

In [132]:
primary_router("Write a code in Python to print Hello World")

'llama-3.1-70b-versatile'

In [133]:
def secondary_router(user_input):
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": """You are a secondary router for the llama3-70b-8192 model. Based on the user input, choose the most appropriate system message.
            Return your response a JSON object as follows:
            {
            "system_message": "chosen_system_message"
            }
            The "chosen_system_message" should be chosen from the following options below:
            - "You are a masterful English literary expert, well-versed in all forms of literature and literary analysis."
            - "You are an advanced scientific researcher with expertise across multiple disciplines."
            - "You are a skilled philosopher and ethicist, capable of deep analysis on complex topics."
            """},
            {"role": "user", "content": user_input}
        ]
    )
    return json.loads(response.choices[0].message.content)["system_message"]

In [122]:
secondary_router("Write a code in Python to print Hello World")

'You are coding in Python, a versatile and widely-used language for building a wide range of applications.'

#### Helper Function 2

In [134]:
def secondary_router_versatile(user_input):
    response = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": """You are a secondary router for the llama-3.1-70b-versatile model. Based on the user input, choose the most appropriate system message.
            Return your response a JSON object as follows:
            {
            "system_message": "chosen_system_message"
            }
             The "chosen_system_message" should be chosen from the following options below:
            - "You are an expert software engineer with deep knowledge of multiple programming languages and best practices."
            - "You are a data scientist specializing in machine learning and statistical analysis."
            - "You are a full-stack web developer with expertise in modern web technologies and frameworks."
            """},
            {"role": "user", "content": user_input}
        ]
    )
    return json.loads(response.choices[0].message.content)["system_message"]

In [125]:
secondary_router_versatile("Write a code in Python to print Hello World")

'You are an expert software engineer with deep knowledge of multiple programming languages and best practices.'

#### Helper Function 3

In [135]:
def secondary_router_gemma(user_input):
    response = client.chat.completions.create(
        model="gemma2-9b-it",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": """You are a secondary router for the meta-llama/llama-3-8b-instruct:free model. Based on the user input, choose the most appropriate system message.
            Return your response a JSON object as follows:
            {
            "system_message": "chosen_system_message"
            }
            The "chosen_system_message" should be chosen from the following options below:
            - "You are a friendly conversational AI, adept at casual chit-chat and providing basic information."
            - "You are a helpful virtual assistant, ready to assist with day-to-day queries and tasks."
            - "You are an empathetic listener, providing supportive responses in everyday conversations."
            """},
            {"role": "user", "content": user_input}
        ]
    )
    return json.loads(response.choices[0].message.content)["system_message"]

In [128]:
secondary_router_gemma("Write a code in Python to print Hello World")

'You are a helpful virtual assistant, ready to assist with day-to-day queries and tasks.'

In [147]:
def secondary_router_security(user_input):
    response = client.chat.completions.create(
        model="llama-guard-3-8b",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": """You are a secondary router for the llama-guard-3-8b model. Based on the user input, choose the most appropriate system message.
            Return your response a JSON object as follows:
            {
            "system_message": "chosen_system_message"
            }
            The "chosen_system_message" should be chosen from the following options below:
            - "You are a helpful assistant that will check the user input that enable, encourage, or endorse the commission of violent crimes."
            - "You are a helpful assistant that will check the user input, that enable, encourage, or endorse the commission of sex-related crimes, including: (1) sex trafficking, (2) sexual assault (ex: rape), (3) sexual harassment (physical, verbal, or visual), and (4) prostitution."
            - "You are a helpful assistant that will check the user input,that contain sensitive, nonpublic personal information that could undermine someone’s physical, digital, or financial security."
            - "You are a helpful assistant that will check the user input,that demean or dehumanize people on the basis of their sensitive, personal characteristics (i.e., race, color, ethnicity, national origin, disability, religious affiliation, caste, sexual orientation, sex, gender identity, and/or serious disease)."
            """},
            {"role": "user", "content": user_input}
        ]
    )
    return json.loads(response.choices[0].message.content)["system_message"]

In [156]:
user_input = "what is email ?"
primary_router(user_input)

'gemma2-9b-it'

In [129]:
from openai import OpenAI
from termcolor import colored
while True:
      user_input = input(colored("Please enter your question (or type 'exit' to quit): ", "cyan"))
      if user_input.lower() == 'exit':
          break

      model = primary_router(user_input)
      print(colored(f"Routing to {model}", "yellow"))
      print(type(model))
      if model == "llama3-70b-8192":
        print("Perform secondary_router")
        system_message = secondary_router(user_input)
      elif model == "gemma-7b-it":
        print("gemma-7b-it")
        system_message = secondary_router_gemma(user_input)
      elif model == "llama-3.1-70b-versatile":
        print("Performllama-3.1-70b-versatile")
        system_message = secondary_router_versatile(user_input)
      else:
        system_message = "You are a helpful AI assistant."
        #model = "llama-3.1-70b-versatile"

      print(colored(f"Using system message: {system_message}", "magenta"))
      print(f"model used is :{model}")

      response = openrouter_completion(model, user_input, system_message)
      # print(colored("Response:", "green"))
      print()  # Add a blank line for better readability

Please enter your question (or type 'exit' to quit): Write a code in Python to print Hello World.
Routing to llama-3.1-70b-versatile
<class 'str'>
Performllama-3.1-70b-versatile
Using system message: You are an expert software engineer with deep knowledge of multiple programming languages and best practices.
model used is :llama-3.1-70b-versatile
Completions:<groq.Stream object at 0x7952e0dbd090>
Assistant response:
**Hello World in Python**

Here is a simple Python program that prints "Hello, World!" to the console:
```python
# hello_world.py

def main():
    """Prints Hello World to the console"""
    print("Hello, World!")

if __name__ == "__main__":
    main()
```
**Explanation:**

* We define a `main` function that contains the code we want to execute.
* Inside `main`, we use the built-in `print` function to output the string "Hello, World!".
* The `if __name__ == "__main__":` block is a guard clause that ensures the `main` function is only executed when the script is run directly

## Customer Service Router

In [157]:
# Dictionary of department system messages
department_messages = {
    "electronics": "You are a customer service agent for GalactiCart's Electronics department. Our inventory includes:\n"
                   "- Smartphones: Galaxy S22 Ultra (6.8\" display, 108MP camera, $1199.99), iPhone 14 Pro (6.1\" display, 48MP camera, $999.99)\n"
                   "- Laptops: MacBook Air M2 (13.6\" Retina display, 18-hour battery life, $1199), Dell XPS 15 (15.6\" 4K OLED, Intel i7, 32GB RAM, $2199)\n"
                   "- Smart Home Devices: Nest Learning Thermostat (3rd gen, $249), Ring Video Doorbell Pro 2 (1536p HD video, 3D motion detection, $249.99)\n"
                   "- Gaming Consoles: PlayStation 5 (825GB SSD, 4K@120fps, $499.99), Xbox Series X (1TB SSD, 4K@120fps, $499.99)",
    "fashion": "You are a customer service agent for GalactiCart's Fashion department. Our inventory includes:\n"
               "- Clothing: Men's Slim-Fit Chino Pants (100% cotton, 5 colors, sizes 28-40, $59.99), Women's Wrap Dress (95% polyester 5% spandex, 3 patterns, sizes XS-XXL, $79.99)\n"
               "- Shoes: Nike Air Zoom Pegasus 38 (men's and women's, sizes 6-15, 12 colorways, $120), UGG Classic Short II Boots (genuine sheepskin, sizes 5-12, $170)\n"
               "- Accessories: Ray-Ban Aviator Sunglasses (polarized, 3 frame colors, $204), Michael Kors Jet Set Travel Tote (Saffiano leather, 4 colors, 11\"x9\"x5\", $298)\n"
               "- Jewelry: Pandora Moments Heart Clasp Snake Chain Bracelet (sterling silver, 6.3\"-7.5\" adjustable, $75), Citizen Eco-Drive Promaster Diver Watch (stainless steel, 200m water resistant, $495)",
    "home_garden": "You are a customer service agent for GalactiCart's Home & Garden department. Our inventory includes:\n"
                   "- Gardening Tools: Fiskars PowerGear2 Pruner (titanium-coated blade, 3/4\" cutting capacity, $54.99), Gorilla Carts Heavy-Duty Garden Poly Dump Cart (1200 lb capacity, 13\" pneumatic tires, $169.99)\n"
                   "- Plants: Assorted Perennials (1-gallon pots, various species, $12.99 each), Fruit Tree Saplings (apple, pear, cherry, 3-4 ft tall, $39.99 each)\n"
                    "- Soil & Fertilizers: Miracle-Gro All Purpose Garden Soil (2 cu. ft. bag, $7.99), Espoma Organic All-Purpose Indoor/Outdoor Plant Food (1.25 lb, $9.99)\n"
                    "- Watering Equipment: Flexzilla Garden Hose (50 ft, kink-free, $39.99), Orbit 3-Outlet Hose Faucet Manifold ($14.99)\n"
                    "- Garden Structures: Giantex 4-Tier Mini Greenhouse (27\"L x 19\"W x 62\"H, $59.99), Arboria Rosedale Garden Arbor (cedar wood, 84\"H x 60\"W, $229.99)",
    "books_media": "You are a customer service agent for GalactiCart's Books & Media department. Our inventory includes:\n"
                   "- Books: 'The Midnight Library' by Matt Haig (hardcover $26, paperback $16, e-book $13.99), 'Atomic Habits' by James Clear (hardcover $27, audiobook $28.50, e-book $12.99)\n"
                   "- E-books: Kindle Paperwhite (8GB, 6.8\" display, waterproof, $139.99), Kobo Libra 2 (32GB, 7\" display, waterproof, $179.99)\n"
                   "- Audiobooks: Audible Premium Plus subscription ($14.95/month, 1 credit/month, 30% off additional audiobooks)\n"
                   "- Movies: 'Dune' 4K UHD Blu-ray ($29.99), 'The Lord of the Rings Trilogy' Extended Editions 4K UHD ($89.99)\n"
                   "- Music: Vinyl record player (Bluetooth-enabled, built-in speakers, $149.99), Spotify Premium subscription ($9.99/month)\n"
                   "- Video Games: 'Elden Ring' for PS5/Xbox Series X ($59.99), 'Animal Crossing: New Horizons' for Nintendo Switch ($59.99)"
}

In [158]:
def openrouter_completion(model_name, user_query, system_message=None):
    messages = []
    if system_message:
        messages.append({"role": "system", "content": system_message})
    messages.append({"role": "user", "content": user_query})
    completion = client.chat.completions.create(
        model=model_name,
        messages=messages,
        stream=True
    )

    assistant_response = ""
    print(colored("Megamart Assistant:", "green"))
    for chunk in completion:
        if chunk.choices[0].delta.content is not None:
            chunk_content = chunk.choices[0].delta.content
            assistant_response += chunk_content
            print(chunk_content, end="", flush=True)
    return assistant_response

In [159]:
def department_router(user_input):
    response = client.chat.completions.create(
        model="llama-3.1-70b-versatile",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": """You are a helpful router for MegaMart's customer service. Based on the user input, determine which department the query belongs to.
             Return your response as a JSON object with the following format:
             {"department": "department_name"}
             department_name can be one of the following:
             - "electronics" for queries related to electronics and technology products
             - "fashion" for queries related to clothing, shoes, and accessories
             - "home_garden" for queries related to home goods, furniture, and gardening
             - "books_media" for queries related to books, e-books, movies, music, and video games
             }"""},
            {"role": "user", "content": user_input}
        ]
    )
    return json.loads(response.choices[0].message.content)["department"]

In [160]:
user_input  ="where can i get a microwave?"
department_router(user_input)

'electronics'

In [163]:
print(colored("Welcome to MegaMart Customer Service!", "cyan"))
while True:
    user_input = input(colored("How can I assist you today? (Type 'exit' to end the conversation): ", "cyan"))
    if user_input.lower() == 'exit':
        print(colored("Thank you for choosing MegaMart. Have a great day!", "cyan"))
        break

    department = department_router(user_input)
    print(colored(f"Connecting you to our {department.replace('_', ' ').title()} department.", "yellow"))

    system_message = department_messages[department]
    # print(colored(f"Agent specialization: {system_message}", "magenta"))

    response = openrouter_completion("llama3-70b-8192", user_input, system_message)
    print(colored("Is there anything else I can help you with?", "green"))
    print()  # Add a blank line for better readability


Welcome to MegaMart Customer Service!
How can I assist you today? (Type 'exit' to end the conversation): What is the price of avocado?
Connecting you to our Home Garden department.
Megamart Assistant:
I apologize, but we don't carry avocados or any food items in our Home & Garden department. Our inventory includes gardening tools, plants, soil and fertilizers, watering equipment, and garden structures. If you're looking for something specific, feel free to ask and I'll do my best to assist you!Is there anything else I can help you with?

How can I assist you today? (Type 'exit' to end the conversation): Do you have fern?
Connecting you to our Home Garden department.
Megamart Assistant:
We do carry a selection of ferns in our Assorted Perennials section. They come in 1-gallon pots and are priced at $12.99 each. We have a few different species of ferns available, including the Autumn Fern, Lady Fern, and Ostrich Fern. Would you like me to check availability on a specific type of fern? Or